
# <center>Predicting the returns of orders  for a retail shoe seller</center>




## Introduction
###  Challenge SD210 2018
#### Authors :  Florence D'Alché & Umut Şimşekli & Moussab Djerrab


**Context of the challenge:**

An electronic commerce company sells shoes, and has a high return rate of his products, more than 20%. This large number of returns and exchanges has a negative impact on its margin. To remedy this problem, the company wants to better understand this phenomenon, and have tools to quantify the probability of return for a given product. It makes available its database of orders placed between October 2011 and October 2015, its product feedback data, and its customer and product databases (provide the data dictionary).

**Goal of the challenge:**
<ul>
<li>Identify conditions that favor product return (eg what type of product is usually returned, which customer is more keen on returning a product, what type of order or purchase context most often leads to returns?)</li>
<li>Build a return forecast template for each product from a shopping cart.
</li>
</ul>

To go further: This project aimes at making stand out purchasing behaviors. With this knowledge, the e-merchant wishes to use this data to better plan his activity. In particular, he wants to forecast the turnover generated by his clients.



**Training data:**

There will be $N= 1067290$ lines of orders in the training dataset. For each order  the training dataset reports if the command has been returned (***ReturnQuantityBin***) and the quantity returned (***ReturnQuantity***). The column to target (***ReturnQuantityBin***) which is a binary column ($y = 1$ if returned and $y=0$ otherwise). 

**Test data:**

The test data contain $N_\text{test} = 800468$ lines of orders. Everything else is similar to the training data.


## Additional Data

As part of the challenge, two additional datasets are avalaible namely (**customers.csv**) and (**products.csv**). Those to sets contains informations on custmers and on the products. A good prediction model will necessarily require extraction of information comming from this dataset. Students are free to use these data as they see fit. Please keep in mind that both sets containes also customers and products that are not present in the training or test sets.

A dictionnary of variables (**dictionnary.xlsx**) is avalaible in the folder containing the datasets. Please refer to it so as to have a definition of the variables at hand.


## The goal and the performance criterion

In this challenge, we will use an evaluation metric, which is commonly used in binary prediction, namely the ROC AUC criteria. **The closest to 1 the better (be affarait if its below 0).**
Hence the form of the file to send is of the form :


| <center> probability </center>  |
| ------------- |
| <center> .90  </center>         |
| <center> ...  </center>         |
| <center> .42  </center>         |


The order of the probabilities needs to respect the order in the test set.



# Training Data

https://www.dropbox.com/sh/uo4oudw43j45mp3/AACA0UqkitNKSWdE_7fs2Wbla?dl=0


In [3]:
from datetime import datetime
from __future__ import division
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import sys

%matplotlib inline

## Data loading

In [16]:
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
orders = pd.read_csv("X_train.csv")
orders_label   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

## Data exploration
Since we dispose of three datasets, two of which are huge in terms of columns, it would be preferrable to give a first look at them separately.

In [17]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,14089083.0,SE,1979-02-05 00:00:00,Femme,2013-03-16 23:00:05
1,12862066.0,FR,1982-08-04 00:00:00,Femme,2012-02-14 17:47:33
2,14791699.0,FR,1965-04-02 00:00:00,Femme,2013-10-04 23:10:42
3,10794664.0,FR,1966-04-09 00:00:00,Femme,2010-03-25 18:46:59
4,15268576.0,ES,1980-04-22 00:00:00,Femme,2014-03-19 10:48:39


In [18]:
customers.dtypes

CustomerId        float64
CountryISOCode     object
BirthDate          object
Gender             object
FirstOrderDate     object
dtype: object

In [29]:
customers.isnull().sum()

CustomerId        0
CountryISOCode    0
BirthDate         0
Gender            0
FirstOrderDate    0
dtype: int64

In [20]:
customers.describe() # useless

,CustomerId
count,7.802380e+05
mean,1.458469e+07
std,2.078408e+06
min,1.000001e+07
25%,1.303418e+07
50%,1.486903e+07
75%,1.632396e+07
max,1.788973e+07


In [35]:
customers_contvars = customers.describe().columns
customers_contvars

Index(['CustomerId'], dtype='object')

In [36]:
customers_catvars = customers.describe(include=["object"]).columns
customers_catvars

Index(['CountryISOCode', 'BirthDate', 'Gender', 'FirstOrderDate'], dtype='object')

### TODO:
- Set also id as categorical through label encoding, in order not to allow computation on it as if it were a number. We could also directly exclude it from analysis, since it doesn't add value anyway
- Date manipulation: 'BirthDate' and 'FirstOrderDate' should be transformed in columns counting days: birthdate becomes age column and the other becomes something like 'days_from_firstoder' blabla. Last date to be considered is October 2015.

In [21]:
products.head()

,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,728257.0,Homme,Classique,Automne/Hiver,Saisonnier,66.0,DÃ©tente,Standard,17267.0,Baskets,...,NaN,0.0,30.50,0.0,Montantes,NaN,NaN,NaN,False,Prenez une taille en dessous de votre pointure...
1,806356.0,Femme,ND,Automne/Hiver,Saisonnier,842.0,ND,Standard,30824.0,Baskets,...,0.0,0.0,43.00,0.0,Montantes,NaN,NaN,NaN,True,Prenez votre pointure habituelle
2,768790.0,Femme,ND,Automne/Hiver,Reconduit,988.0,Ville,Standard,62475.0,Bottines et boots,...,6.0,3.0,54.90,0.0,Bout pointu,NaN,NaN,NaN,False,Prenez votre pointure habituelle
3,515679.0,Femme,ND,Automne/Hiver,Saisonnier,769.0,Ville,Standard,43983.0,Escarpins,...,0.0,13.0,34.50,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle
4,1025246.0,Femme,ND,Automne/Hiver,Saisonnier,1244.0,ND,Standard,81493.0,Bottines et boots,...,8.0,4.0,43.76,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle


In [22]:
products.dtypes

VariantId                float64
GenderLabel               object
MarketTargetLabel         object
SeasonLabel               object
SeasonalityLabel          object
BrandId                  float64
UniverseLabel             object
TypeBrand                 object
ProductId                float64
ProductType               object
SupplierColor             object
ProductColorId           float64
MinSize                  float64
MaxSize                  float64
CalfTurn                 float64
UpperHeight              float64
HeelHeight               float64
PurchasePriceHT          float64
IsNewCollection          float64
SubtypeLabel              object
UpperMaterialLabel        object
LiningMaterialLabel       object
OutSoleMaterialLabel      object
RemovableSole             object
SizeAdviceDescription     object
dtype: object

In [30]:
products.isnull().sum()

VariantId                     0
GenderLabel                   0
MarketTargetLabel             0
SeasonLabel                   0
SeasonalityLabel              0
BrandId                       0
UniverseLabel                 0
TypeBrand                     0
ProductId                     0
ProductType                1102
SupplierColor                 0
ProductColorId                0
MinSize                       0
MaxSize                       0
CalfTurn                 469539
UpperHeight              413501
HeelHeight                21625
PurchasePriceHT               0
IsNewCollection               0
SubtypeLabel              74305
UpperMaterialLabel       465587
LiningMaterialLabel      525811
OutSoleMaterialLabel     526199
RemovableSole             23107
SizeAdviceDescription     36301
dtype: int64

### TODO: 
Inspect columns with nulls to understand if nulls are a proxy for 0 or if it's simply missing data.
In the first case, substitute with 0. Otherwise, we should fill data... KNN classifier? (ask Pietro) 

In [32]:
products.describe()

,VariantId,BrandId,ProductId,ProductColorId,MinSize,MaxSize,CalfTurn,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection
count,5.335200e+05,533520.000000,533520.000000,533520.000000,533520.000000,533520.000000,63981.000000,120019.000000,511895.000000,533520.000000,533520.000000
mean,1.038272e+06,488.597455,74767.941378,149110.895756,40.367902,40.450424,7.944455,7.994525,2.432194,38.202530,0.059563
std,3.478620e+05,385.038946,29038.845180,51465.508400,26.744381,26.850812,15.089255,9.845346,3.474408,23.165556,0.236675
min,9.020000e+02,3.000000,36.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.091625e+05,122.000000,52245.000000,100020.750000,36.000000,36.000000,0.000000,0.000000,0.000000,24.000000,0.000000
50%,1.052074e+06,439.000000,71174.000000,151232.500000,39.000000,39.000000,0.000000,5.000000,0.000000,34.099998,0.000000
75%,1.326417e+06,818.000000,98331.000000,194758.000000,41.000000,41.000000,0.000000,10.000000,4.000000,46.850000,0.000000
max,1.713682e+06,1374.000000,128354.000000,242150.000000,720.000000,720.000000,60.000000,58.000000,113.000000,1100.000000,1.000000


In [33]:
products_contvars = products.describe().columns
products_contvars

Index(['VariantId', 'BrandId', 'ProductId', 'ProductColorId', 'MinSize',
       'MaxSize', 'CalfTurn', 'UpperHeight', 'HeelHeight', 'PurchasePriceHT',
       'IsNewCollection'],
      dtype='object')

In [34]:
products_catvars = products.describe(include=["object"]).columns
products_catvars

Index(['GenderLabel', 'MarketTargetLabel', 'SeasonLabel', 'SeasonalityLabel',
       'UniverseLabel', 'TypeBrand', 'ProductType', 'SupplierColor',
       'SubtypeLabel', 'UpperMaterialLabel', 'LiningMaterialLabel',
       'OutSoleMaterialLabel', 'RemovableSole', 'SizeAdviceDescription'],
      dtype='object')

### TODO:
- All columns with ID are not continuous but categorical, for some reason are considered as floats! Also col 'IsNewCollection' is categorical (0 or 1)
- Column 'SizeAdviceDescription' seems veeeery interesting for analysis, but we should find a way to extract values from it...

In [25]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,DeviceTypeLabel,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,1,1,"13,17",2011-10-26 12:11:38,2011-10-26 17:48:00,1,0.0,77700
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"56,4",2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,37,2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600


In [26]:
orders.dtypes

OrderNumber            int64
VariantId              int64
LineItem               int64
CustomerId             int64
OrderStatusLabel      object
OrderTypelabel        object
SeasonLabel           object
PayementModeLabel     object
CustomerTypeLabel     object
IsoCode               object
DeviceTypeLabel       object
PricingTypeLabel      object
TotalLineItems         int64
Quantity               int64
UnitPMPEUR            object
OrderCreationDate     object
OrderShipDate         object
OrderNumCustomer       int64
IsOnSale             float64
BillingPostalCode     object
dtype: object

In [31]:
orders.isnull().sum()

OrderNumber             0
VariantId               0
LineItem                0
CustomerId              0
OrderStatusLabel        0
OrderTypelabel          0
SeasonLabel             0
PayementModeLabel       0
CustomerTypeLabel       0
IsoCode                 0
DeviceTypeLabel         0
PricingTypeLabel        0
TotalLineItems          0
Quantity                0
UnitPMPEUR              0
OrderCreationDate       0
OrderShipDate           0
OrderNumCustomer        0
IsOnSale             7815
BillingPostalCode      52
dtype: int64

### TODO: 
- No idea of how to fill missing BillingPostalCode. Suggestions appreciated! ^^
- I would probably fill IsOnSale nulls with 0, meaning they were not on sale, but I'm not 100% sure. In the end however, column should be categorical

In [28]:
orders.describe()

,OrderNumber,VariantId,LineItem,CustomerId,TotalLineItems,Quantity,OrderNumCustomer,IsOnSale
count,1.067290e+06,1.067290e+06,1.067290e+06,1.067290e+06,1.067290e+06,1.067290e+06,1.067290e+06,1.059475e+06
mean,4.998603e+07,6.198179e+05,1.449695e+00,1.254391e+07,1.912211e+00,1.003925e+00,3.873052e+00,3.006102e-01
std,2.892062e+07,2.153218e+05,1.074525e+00,1.314315e+06,1.859919e+00,1.049004e-01,1.235264e+01,4.585236e-01
min,5.100000e+01,9.020000e+02,1.000000e+00,1.000001e+07,1.000000e+00,1.000000e+00,-1.000000e+00,0.000000e+00
25%,2.475954e+07,4.873360e+05,1.000000e+00,1.137852e+07,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
50%,4.992306e+07,6.405210e+05,1.000000e+00,1.276514e+07,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00
75%,7.517581e+07,7.774260e+05,2.000000e+00,1.361293e+07,2.000000e+00,1.000000e+00,4.000000e+00,1.000000e+00
max,9.999179e+07,1.161135e+06,3.900000e+01,1.467394e+07,2.000000e+02,3.000000e+01,4.430000e+02,1.000000e+00


In [37]:
orders_contvars = orders.describe().columns
orders_contvars

Index(['OrderNumber', 'VariantId', 'LineItem', 'CustomerId', 'TotalLineItems',
       'Quantity', 'OrderNumCustomer', 'IsOnSale'],
      dtype='object')

In [38]:
orders_catvars = orders.describe(include=["object"]).columns
orders_catvars

Index(['OrderStatusLabel', 'OrderTypelabel', 'SeasonLabel',
       'PayementModeLabel', 'CustomerTypeLabel', 'IsoCode', 'DeviceTypeLabel',
       'PricingTypeLabel', 'UnitPMPEUR', 'OrderCreationDate', 'OrderShipDate',
       'BillingPostalCode'],
      dtype='object')

## Defining a feature transformation

In [3]:
def funk_mask(d):
    " Defining a simple mask over the input data "
    columns_ext = ["OrderCreationDate","OrderNumber","VariantId", "CustomerId","OrderCreationDate","OrderShipDate","BillingPostalCode"]
    X1 = d.loc[:,[xx for xx in d.columns if xx not in columns_ext]]
    g = lambda x: x.replace(",",".")
    X1.UnitPMPEUR = map(np.float64,(map(g,X1.UnitPMPEUR)))
    columns2bin = [x for x in X1.columns if X1[x].dtype == np.dtype('O')]
    X2 = pd.get_dummies(X1.loc[:,columns2bin])
    X1 = X1.loc[:,[xx for xx in X1.columns if xx not in columns2bin]]
    res = pd.concat([X1,X2],axis=1)
    res = res.fillna(0)
    return(res)

## Applying the mask

In [4]:
x1 = funk_mask(X_train)
x2 = funk_mask(X_test)
seleckt_columns = np.intersect1d(x1.columns,x2.columns)
x1 = x1.loc[:,seleckt_columns]
x2 = x2.loc[:,seleckt_columns]

## Supervised learning : Logistic regression model

In [12]:
clf = LogisticRegression()
clf.fit(x1.iloc[:50000], y_train.ReturnQuantityBin[:50000])
y_tosubmit = clf.predict_proba(x2.loc[:,x1.columns])

## Score of our prediction : on the train

In [13]:
yres = clf.predict_proba(x1.loc[:100000,x1.columns])
roc_auc_score(y_train.ReturnQuantityBin.iloc[:100001],yres[:,1])

# Submission to the system
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')


# <center> That's all folks; Good Luck! </center>